## Model Registry

In [2]:
from focoos.model_registry import ModelRegistry

registry = ModelRegistry()
print(registry.list_models())


model_info = registry.get_model_info("fai-detr-l-obj365")

model_info.pprint()

['fai-detr-l-obj365', 'fai-detr-l-coco', 'fai-detr-m-coco', 'fai-detr-s-coco', 'fai-detr-n-coco', 'fai-mf-l-ade', 'fai-mf-m-ade', 'fai-mf-l-coco-ins', 'fai-mf-m-coco-ins', 'fai-mf-s-coco-ins', 'bisenetformer-m-ade']


[05/22 07:51][INFO][model_info]: 
            📋 Name: fai-detr-l-obj365
            📝 Description: RTDETR Large model (Object365)
            👪 Family: ModelFamily.DETR
            🔗 Focoos Model: fai-detr-l-obj365
            🎯 Task: Task.DETECTION
            🏷️ Classes: ['Person', 'Sneakers', 'Chair', 'Other Shoes', 'Hat', 'Car', 'Lamp', 'Glasses', 'Bottle', 'Desk', 'Cup', 'Street Lights', 'Cabinet/shelf', 'Handbag/Satchel', 'Bracelet', 'Plate', 'Picture/Frame', 'Helmet', 'Book', 'Gloves', 'Storage box', 'Boat', 'Leather Shoes', 'Flower', 'Bench', 'Potted Plant', 'Bowl/Basin', 'Flag', 'Pillow', 'Boots', 'Vase', 'Microphone', 'Necklace', 'Ring', 'SUV', 'Wine Glass', 'Belt', 'Moniter/TV', 'Backpack', 'Umbrella', 'Traffic Light', 'Speaker', 'Watch', 'Tie', 'Trash bin Can', 'Slippers', 'Bicycle', 'Stool', 'Barrel/bucket', 'Van', 'Couch', 'Sandals', 'Bakset', 'Drum', 'Pen/Pencil', 'Bus', 'Wild Bird', 'High Heels', 'Motorcycle', 'Guitar', 'Carpet', 'Cell Phone', 'Bread', 'Camera', 'Canned

## AutoDataset

In [6]:
from focoos.data.auto_dataset import AutoDataset
from focoos.data.default_aug import get_default_by_task
from focoos.ports import DatasetLayout, DatasetSplitType, Task

task = Task.DETECTION
layout = DatasetLayout.ROBOFLOW_COCO
auto_dataset = AutoDataset(dataset_name="aquarium", task=task, layout=layout, datasets_dir="../datasets")

train_augs, val_augs = get_default_by_task(task, 640, advanced=False)
train_dataset = auto_dataset.get_split(augs=train_augs.get_augmentations(), split=DatasetSplitType.TRAIN)
valid_dataset = auto_dataset.get_split(augs=val_augs.get_augmentations(), split=DatasetSplitType.VAL)

[05/22 07:54][INFO][focoos.data.auto_dataset]: ✅ Dataset name: aquarium, Dataset Path: ../datasets/aquarium, Dataset Layout: DatasetLayout.ROBOFLOW_COCO
[05/22 07:55][INFO][focoos.data.datasets.dict_dataset]: [Focoos-DictDataset] dataset aquarium loaded. len: 448, classes:7 ,../datasets/aquarium/train
[05/22 07:55][DEBUG][focoos.data.datasets.serialize]: Serializing 448 elements to byte tensors and concatenating them all ...
[05/22 07:55][DEBUG][focoos.data.datasets.serialize]: Serialized dataset takes 0.27 MiB
[05/22 07:55][INFO][focoos.data.mappers.detection_dataset_mapper]: [DatasetMapper] Augmentations used in training: [ColorAugSSDTransform(img_format=RGB, brightness_delta=32, contrast_low=0.5, contrast_high=1.5, saturation_low=0.5, saturation_high=1.5, hue_delta=18), RandomFlip(prob=0.5, horizontal=True, vertical=False), RandomApply(prob=1.0, tfm=Resize(shape=(640, 640))), ResizeShortestEdge(short_edge_length=[452, 905], max_size=800, sample_style=range, interp=2), RandomCrop(cro

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


[05/22 07:55][INFO][focoos.data.datasets.dict_dataset]: [Focoos-DictDataset] dataset aquarium loaded. len: 127, classes:7 ,../datasets/aquarium/valid
[05/22 07:55][DEBUG][focoos.data.datasets.serialize]: Serializing 127 elements to byte tensors and concatenating them all ...
[05/22 07:55][DEBUG][focoos.data.datasets.serialize]: Serialized dataset takes 0.08 MiB
[05/22 07:55][INFO][focoos.data.mappers.detection_dataset_mapper]: [DatasetMapper] Augmentations used in inference: [RandomApply(prob=1.0, tfm=Resize(shape=(640, 640))), ResizeShortestEdge(short_edge_length=[640, 640], max_size=4096, sample_style=range, interp=2)]


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


## Auto Model

In [4]:
from focoos.model_manager import ModelManager

model = ModelManager.get("fai-detr-m-coco", num_classes=train_dataset.dataset.metadata.num_classes)

[05/22 07:51][INFO][ModelManager]: Downloading weights from remote URL: https://public.focoos.ai/pretrained_models/fai-detr-m-coco/model_final.pth
[05/22 07:51][DEBUG][HUB]: 📥 File already exists: /root/FocoosAI/models/fai-detr-m-coco/model_final.pth
[05/22 07:51][WARNING][Checkpoint]: Skip loading parameter 'head.criterion.empty_weight' to the model due to incompatible shapes: (81,) in the checkpoint but (8,) in the model! You might want to double check if this is expected.
[05/22 07:51][WARNING][Checkpoint]: Skip loading parameter 'head.predictor.enc_score_classifier.weight' to the model due to incompatible shapes: (80, 256) in the checkpoint but (7, 256) in the model! You might want to double check if this is expected.
[05/22 07:51][WARNING][Checkpoint]: Skip loading parameter 'head.predictor.enc_score_classifier.bias' to the model due to incompatible shapes: (80,) in the checkpoint but (7,) in the model! You might want to double check if this is expected.
[05/22 07:51][WARNING][Che

## Train with a dataset downloaded from HUB

In [7]:
from focoos.data.auto_dataset import AutoDataset
from focoos.data.default_aug import get_default_by_task
from focoos.hub.focoos_hub import FocoosHUB
from focoos.model_manager import ModelManager
from focoos.ports import DEV_API_URL, DatasetLayout, DatasetSplitType, RuntimeType, Task, TrainerArgs

hub = FocoosHUB(host_url=DEV_API_URL)
my_datasets = hub.list_remote_datasets(include_shared=False)
remote_dataset = hub.get_remote_dataset(my_datasets[6].ref)
dataset_path = remote_dataset.download_data()
auto_dataset = AutoDataset(dataset_name=dataset_path, task=remote_dataset.task, layout=remote_dataset.layout)

train_augs, val_augs = get_default_by_task(remote_dataset.task, 640, advanced=False)
train_dataset = auto_dataset.get_split(augs=train_augs.get_augmentations(), split=DatasetSplitType.TRAIN)
valid_dataset = auto_dataset.get_split(augs=val_augs.get_augmentations(), split=DatasetSplitType.VAL)


model = ModelManager.get("fai-detr-m-coco", num_classes=train_dataset.dataset.metadata.num_classes)

args = TrainerArgs(
    run_name=f"{remote_dataset.name}-{model.model_info.name}",
    output_dir="./experiments",
    amp_enabled=True,
    batch_size=16,
    max_iters=500,
    eval_period=50,
    learning_rate=0.0001,
    scheduler="MULTISTEP",
    weight_decay=0.0001,
    workers=16,
)


model.train(args, train_dataset, valid_dataset)
infer = model.export(runtime_type=RuntimeType.TORCHSCRIPT_32)
infer.benchmark()

[05/22 08:03][ERROR][HUB]: API key is required 🤖


ValueError: API key is required 🤖

## Validation only

In [ ]:
from focoos.ports import TrainerArgs

args = TrainerArgs(
    run_name="aquarium2",
    output_dir="./experiments",
    amp_enabled=True,
    batch_size=16,
    max_iters=300,
    eval_period=100,
    learning_rate=0.0001,
    scheduler="MULTISTEP",
    weight_decay=0.0001,
    workers=16,
)

model.test(args, valid_dataset)

# Detection

In [ ]:
# Use the model for inference
from PIL import Image

from focoos.data.auto_dataset import AutoDataset
from focoos.data.default_aug import get_default_by_task
from focoos.model_manager import ModelManager
from focoos.ports import DatasetLayout, DatasetSplitType, Task, TrainerArgs

task = Task.DETECTION
layout = DatasetLayout.ROBOFLOW_COCO
auto_dataset = AutoDataset(dataset_name="aquarium", task=task, layout=layout)
resolution = 640

train_augs, val_augs = get_default_by_task(task, resolution, advanced=False)
train_dataset = auto_dataset.get_split(augs=train_augs.get_augmentations(), split=DatasetSplitType.TRAIN)
valid_dataset = auto_dataset.get_split(augs=val_augs.get_augmentations(), split=DatasetSplitType.VAL)


model = ModelManager.get("fai-detr-m-coco", num_classes=train_dataset.dataset.metadata.num_classes)

args = TrainerArgs(
    run_name="exp1",
    output_dir="./experiments",
    amp_enabled=True,
    batch_size=16,
    max_iters=100,
    eval_period=100,
    learning_rate=0.0001,
    scheduler="MULTISTEP",
    weight_decay=0.0001,
    workers=16,
)

model.train(args, train_dataset, valid_dataset)

image = Image.open("image.jpg")

outputs = model(image)

print(outputs)

## Image Classification

In [ ]:
# Use the model for inference
from PIL import Image

from focoos.data.auto_dataset import AutoDataset
from focoos.data.default_aug import get_default_by_task
from focoos.model_manager import ConfigManager, ModelManager
from focoos.nn.backbone.resnet import ResnetConfig
from focoos.ports import (
    DatasetLayout,
    DatasetSplitType,
    ModelFamily,
    ModelInfo,
    Task,
    TrainerArgs,
)

task = Task.CLASSIFICATION
layout = DatasetLayout.CLS_FOLDER
auto_dataset = AutoDataset(dataset_name="hymenoptera", task=task, layout=layout)
resolution = 224

train_augs, val_augs = get_default_by_task(task, resolution, advanced=False)
train_dataset = auto_dataset.get_split(augs=train_augs.get_augmentations(), split=DatasetSplitType.TRAIN)
valid_dataset = auto_dataset.get_split(augs=val_augs.get_augmentations(), split=DatasetSplitType.VAL)


# Create a configuration with a ResNet backbone
cls_config = ConfigManager.from_dict(
    ModelFamily.IMAGE_CLASSIFIER,
    {
        "backbone_config": dict(ResnetConfig(model_type="resnet", depth=50, pretrained=True)),
        "num_classes": valid_dataset.dataset.metadata.num_classes,
        "resolution": resolution,
        "hidden_dim": 512,
        "dropout_rate": 0.2,
    },
)

model_info = ModelInfo(
    name="fai-cls-resnet50",
    description="ResNet50 model for classification",
    task=Task.CLASSIFICATION,
    classes=["cat", "dog", "bird"],
    im_size=224,
    model_family=ModelFamily.IMAGE_CLASSIFIER,
    config=cls_config,
)
# Create the model
model = ModelManager.get(name=model_info.name, model_info=model_info)

args = TrainerArgs(
    run_name="footballxyz",
    output_dir="./experiments",
    amp_enabled=True,
    batch_size=16,
    max_iters=50,
    eval_period=100,
    learning_rate=0.0001,
    scheduler="MULTISTEP",
    weight_decay=0.0001,
    workers=16,
)

model.train(args, train_dataset, valid_dataset)

image = Image.open("image.jpg")
outputs = model(image)

print(outputs)

# SEGMENTATION

In [6]:
from PIL import Image

from focoos.data.auto_dataset import AutoDataset
from focoos.data.default_aug import get_default_by_task
from focoos.model_manager import ModelManager
from focoos.ports import DatasetLayout, DatasetSplitType, Task, TrainerArgs

task = Task.SEMSEG
layout = DatasetLayout.ROBOFLOW_SEG
auto_dataset = AutoDataset(dataset_name="pizza", task=task, layout=layout)

train_augs, val_augs = get_default_by_task(task, 640, advanced=False)
train_dataset = auto_dataset.get_split(augs=train_augs.get_augmentations(), split=DatasetSplitType.TRAIN)
valid_dataset = auto_dataset.get_split(augs=val_augs.get_augmentations(), split=DatasetSplitType.VAL)

model = ModelManager.get("bisenetformer-m-ade", num_classes=valid_dataset.dataset.metadata.num_classes)

args = TrainerArgs(
    run_name="footballxyz",
    output_dir="./experiments",
    amp_enabled=True,
    batch_size=16,
    max_iters=50,
    eval_period=100,
    learning_rate=0.0001,
    scheduler="MULTISTEP",
    weight_decay=0.0001,
    workers=16,
)

model.train(args, train_dataset, valid_dataset)

image = Image.open("image.jpg")
outputs = model(image)

# print(outputs.logits.shape,outputs.masks.shape)
for det in outputs[0].detections:
    print(det.cls_id, det.conf)

[05/21 15:16][INFO][focoos.data.auto_dataset]: ✅ Dataset name: pizza, Dataset Path: /root/FocoosAI/datasets/pizza, Dataset Layout: DatasetLayout.ROBOFLOW_SEG


FileNotFoundError: Dataset /root/FocoosAI/datasets/pizza not found

In [ ]:
from PIL import Image

from focoos.data.auto_dataset import AutoDataset
from focoos.data.default_aug import get_default_by_task
from focoos.model_manager import ModelManager
from focoos.ports import DatasetLayout, DatasetSplitType, Task, TrainerArgs

task = Task.INSTANCE_SEGMENTATION
layout = DatasetLayout.ROBOFLOW_COCO
auto_dataset = AutoDataset(dataset_name="fruits", task=task, layout=layout)

train_augs, val_augs = get_default_by_task(task, 640, advanced=False)
train_dataset = auto_dataset.get_split(augs=train_augs.get_augmentations(), split=DatasetSplitType.TRAIN)
valid_dataset = auto_dataset.get_split(augs=val_augs.get_augmentations(), split=DatasetSplitType.VAL)

model = ModelManager.get("fai-mf-s-coco-ins", num_classes=valid_dataset.dataset.metadata.num_classes)

args = TrainerArgs(
    run_name="footballxyz",
    output_dir="./experiments",
    amp_enabled=True,
    batch_size=16,
    max_iters=50,
    eval_period=100,
    learning_rate=0.0001,
    scheduler="MULTISTEP",
    weight_decay=0.0001,
    workers=16,
)

model.train(args, train_dataset, valid_dataset)

image = Image.open("image.jpg")
outputs = model(image)

# print(outputs.logits.shape,outputs.masks.shape)
for det in outputs[0].detections:
    print(det.cls_id, det.bbox, det.conf)

In [ ]:
from focoos.utils.system import get_system_info

get_system_info().pprint()

In [ ]:
from focoos.model_registry import ModelRegistry

registry = ModelRegistry()
print(registry.list_models())

In [ ]:
from focoos.model_manager import ModelManager

model = ModelManager.get("fai-detr-l-obj365")

## Export

In [ ]:
import supervision as sv

from focoos.ports import Task
from focoos.utils.vision import fai_detections_to_sv

# Initialize annotation utilities
label_annotator = sv.LabelAnnotator(text_padding=10, border_radius=10)
box_annotator = sv.BoxAnnotator()
mask_annotator = sv.MaskAnnotator()


def annotate(im, model_info, detections):
    detections = fai_detections_to_sv(detections, im.shape[:2])
    if len(detections.xyxy) == 0:
        print("No detections found, skipping annotation")
        return im
    classes = model_info.classes
    labels = [
        f"{classes[int(class_id)] if classes is not None else str(class_id)}: {confid * 100:.0f}%"
        for class_id, confid in zip(detections.class_id, detections.confidence)  # type: ignore
    ]
    if model_info.task == Task.DETECTION:
        annotated_im = box_annotator.annotate(scene=im.copy(), detections=detections)

        annotated_im = label_annotator.annotate(scene=annotated_im, detections=detections, labels=labels)
    elif model_info.task in [
        Task.SEMSEG,
        Task.INSTANCE_SEGMENTATION,
    ]:
        annotated_im = mask_annotator.annotate(scene=im.copy(), detections=detections)

    return Image.fromarray(annotated_im)

In [ ]:
# Create a comparison table of the benchmarking metrics
import matplotlib.pyplot as plt
from PIL import Image
from tabulate import tabulate

from focoos.model_manager import ModelManager
from focoos.model_registry import ModelRegistry
from focoos.ports import RuntimeType

registry = ModelRegistry()

image = Image.open("image.jpg")

model_name = "fai-detr-s-coco"
# model_name = "fai-mf-m-coco-ins"
# model_name = "fai-mf-m-ade"
# model_name = "bisenetformer-m-ade"

model = ModelManager.get(model_name)

metrics_torch = model.benchmark(iterations=50, size=640)
metrics_torch_inner = model.model.benchmark(iterations=50, size=640)

runtime_type = RuntimeType.TORCHSCRIPT_32
infer = model.export(runtime_type=runtime_type, overwrite=True)
metrics_trt = infer.benchmark(iterations=50, size=640)
metrics_inner_ts = infer.runtime.benchmark(iterations=50, size=640)

runtime_type = RuntimeType.ONNX_CUDA32
infer = model.export(runtime_type=runtime_type, overwrite=True)
metrics_onnx = infer.benchmark(iterations=50, size=640)
metrics_inner_onnx = infer.runtime.benchmark(iterations=50, size=640)


# Create data for the table
headers = ["Runtime", "FPS", "Mean Latency (ms)", "Std Deviation (ms)"]
table_data = [
    ["PyTorch", metrics_torch.fps, metrics_torch.mean, metrics_torch.std],
    ["TorchScript", metrics_trt.fps, metrics_trt.mean, metrics_trt.std],
    ["ONNX CUDA", metrics_onnx.fps, metrics_onnx.mean, metrics_onnx.std],
    ["PyTorch Model", metrics_torch_inner.fps, metrics_torch_inner.mean, metrics_torch_inner.std],
    ["TorchScript Model", metrics_inner_ts.fps, metrics_inner_ts.mean, metrics_inner_ts.std],
    ["ONNX CUDA Model", metrics_inner_onnx.fps, metrics_inner_onnx.mean, metrics_inner_onnx.std],
]

# Display the table using tabulate
print(tabulate(table_data, headers=headers, tablefmt="grid"))

# Optionally, create a bar chart to visualize FPS comparison
runtimes = [row[0] for row in table_data]
fps_values = [row[1] for row in table_data]

plt.figure(figsize=(10, 6))
plt.bar(runtimes, fps_values)
plt.title("FPS Comparison Across Different Runtimes")
plt.xlabel("Runtime")
plt.ylabel("Frames Per Second (FPS)")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

## Training sync to HUB

In [ ]:
# Create a comparison table of the benchmarking metrics
import matplotlib.pyplot as plt
from PIL import Image
from tabulate import tabulate

from focoos.data.auto_dataset import AutoDataset
from focoos.data.default_aug import get_default_by_task
from focoos.hub import FocoosHUB
from focoos.model_manager import ModelManager
from focoos.model_registry import ModelRegistry
from focoos.ports import DEV_API_URL, LOCAL_API_URL, DatasetLayout, DatasetSplitType, RuntimeType, Task, TrainerArgs

hub = FocoosHUB(host_url=LOCAL_API_URL)
# my_datasets = hub.list_remote_datasets(include_shared=False)
# remote_dataset = hub.get_remote_dataset(my_datasets[6].ref)
# dataset_path = remote_dataset.download_data()


auto_dataset = AutoDataset(dataset_name="Carrera_go_red_grey", task=Task.DETECTION, layout=DatasetLayout.ROBOFLOW_COCO)

train_augs, val_augs = get_default_by_task(Task.DETECTION, 640, advanced=False)
train_dataset = auto_dataset.get_split(augs=train_augs.get_augmentations(), split=DatasetSplitType.TRAIN)
valid_dataset = auto_dataset.get_split(augs=val_augs.get_augmentations(), split=DatasetSplitType.VAL)

model = ModelManager.get("fai-detr-l-obj365")

args = TrainerArgs(
    run_name=f"{auto_dataset.name}-{model.model_info.name}-6",
    output_dir="./experiments",
    amp_enabled=True,
    batch_size=16,
    max_iters=1000,
    eval_period=100,
    learning_rate=0.0001,
    scheduler="MULTISTEP",
    weight_decay=0.0001,
    workers=16,
    sync_to_hub=True,
)


model.train(args, train_dataset, valid_dataset, hub)
model.export(runtime_type=RuntimeType.TORCHSCRIPT_32, overwrite=True)
# infer = model.export(runtime_type=RuntimeType.TORCHSCRIPT_32, overwrite=True)
# infer.benchmark()

In [ ]:
from focoos.model_manager import ModelManager
from focoos.ports import RuntimeType

model = ModelManager.get("fai-detr-l-obj365")
model.export(runtime_type=RuntimeType.TORCHSCRIPT_32, overwrite=True)

In [ ]:
from focoos.utils.metrics import parse_metrics

metrics = parse_metrics("/home/ubuntu/focoos/notebooks/experiments/carrera-fai-detr-m-coco/metrics.json")
print(metrics.iterations)
print(metrics.valid_metrics)
print(metrics.train_metrics)
print(metrics.infer_metrics)

In [ ]:
from focoos.model_manager import ModelManager
from focoos.ports import RuntimeType

models_dir = "/home/ubuntu/focoos/notebooks/experiments"
model_name = "haloaimbot"

model = ModelManager.get(model_name, models_dir=models_dir)
infer = model.export(
    runtime_type=RuntimeType.TORCHSCRIPT_32,
    overwrite=True,
    out_dir=f"{models_dir}/{model_name}",
)
infer.benchmark()
infer.infer("./image.jpg")

In [ ]:
from focoos.infer.infer_model import InferModel
from focoos.ports import ModelInfo, RuntimeType

model_dir = "/home/ubuntu/FocoosAI/models/fai-detr-l-obj365"
model_dir = "/home/ubuntu/focoos/notebooks/experiments/Carrera_go_red_grey-fai-detr-l-obj365-6"
model_dir = "/home/ubuntu/focoos/notebooks/experiments/haloaimbot"
model_info = ModelInfo.from_json(f"{model_dir}/model_info.json")
print(model_info.config)
infer = InferModel(model_dir=model_dir, model_info=model_info, runtime_type=RuntimeType.TORCHSCRIPT_32)
infer.infer("./image.jpg")